In [1]:
import os 
import time
 
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import cv2


In [2]:
learning_rate = 0.0001
batch_size = 128
num_epochs = 10


In [3]:
train_ind = torch.arange(0, 48000)
valid_ind = torch.arange(48000,50000)

train_transforms = transforms.Compose([transforms.Resize((70,70)),
                                       transforms.RandomCrop((64,64)),
                                       transforms.ToTensor()])
test_transforms = transforms.Compose([transforms.Resize((70,70)),
                                       transforms.RandomCrop((64,64)),
                                       transforms.ToTensor()])

train_valid = datasets.CIFAR10(root='data', train=True, transform=train_transforms, download=True)
test_dataset = datasets.CIFAR10(root='data',train=False,transform=test_transforms,download=False)
train_dataset = Subset(train_valid,train_ind)
valid_dataset = Subset(train_valid,valid_ind)

train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,num_workers=4,shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,batch_size=batch_size,num_workers=4,shuffle=False)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,num_workers=4,shuffle=False)


Files already downloaded and verified


In [4]:
# Checking the dataset
print('Training Set:')
for images, labels in train_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    break
    
# Checking the dataset
print('\nValidation Set:')
for images, labels in valid_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    break

# Checking the dataset
print('\nTesting Set:')
for images, labels in test_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    break

print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

Training Set:
Image batch dimensions: torch.Size([128, 3, 64, 64])
Image label dimensions: torch.Size([128])

Validation Set:
Image batch dimensions: torch.Size([128, 3, 64, 64])
Image label dimensions: torch.Size([128])

Testing Set:
Image batch dimensions: torch.Size([128, 3, 64, 64])
Image label dimensions: torch.Size([128])
375
16
79


In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=11,stride=4,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self,x):
        out = self.features(x)
        out = self.avgpool(out)
        out = out.view(-1,256*6*6)
        out = self.classifier(out)
        return out
        
        

In [6]:
torch.manual_seed(1)
model = AlexNet(10)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [7]:
def metric_acc(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            proba = net(features)
            _, predicted = torch.max(proba, 1)
            num_examples +=targets.size(0)
            correct_pred += (predicted == targets).sum()
        return correct_pred.float()/num_examples*100

In [8]:
start_time = time.time()
loss_list = []
train_list = []
valid_list = []

for epoch in range(num_epochs):
    for batch_ind,(features, targets) in enumerate(train_loader):
        features = features.to(device)
        targets = targets.to(device)
        
        out = model(features)
        loss = criterion(out, targets)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.item())
        if not batch_ind % 150:
            print (f'Epoch: {epoch+1:03d}/{num_epochs:03d} | '
                   f'Batch {batch_ind:03d}/{len(train_loader):03d} |' 
                   f'Loss: {loss:.4f}')
    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        
        train_acc = metric_acc(model, train_loader)
        valid_acc = metric_acc(model, valid_loader)
        
        print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}\n'
              f'Train ACC: {train_acc:.2f} | Validation ACC: {valid_acc:.2f}')
        
        train_list.append(train_acc)
        valid_list.append(valid_acc)
        
    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')
  
elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

Epoch: 001/010 | Batch 000/375 |Loss: 2.3025
Epoch: 001/010 | Batch 150/375 |Loss: 1.8892
Epoch: 001/010 | Batch 300/375 |Loss: 1.6252
Epoch: 001/010
Train ACC: 39.59 | Validation ACC: 38.60
Time elapsed: 0.38 min
Epoch: 002/010 | Batch 000/375 |Loss: 1.6223
Epoch: 002/010 | Batch 150/375 |Loss: 1.4762
Epoch: 002/010 | Batch 300/375 |Loss: 1.2915
Epoch: 002/010
Train ACC: 50.45 | Validation ACC: 51.05
Time elapsed: 0.76 min
Epoch: 003/010 | Batch 000/375 |Loss: 1.3748
Epoch: 003/010 | Batch 150/375 |Loss: 1.4022
Epoch: 003/010 | Batch 300/375 |Loss: 1.2968
Epoch: 003/010
Train ACC: 58.18 | Validation ACC: 57.05
Time elapsed: 1.15 min
Epoch: 004/010 | Batch 000/375 |Loss: 1.2011
Epoch: 004/010 | Batch 150/375 |Loss: 1.1653
Epoch: 004/010 | Batch 300/375 |Loss: 1.1628
Epoch: 004/010
Train ACC: 63.55 | Validation ACC: 62.00
Time elapsed: 1.54 min
Epoch: 005/010 | Batch 000/375 |Loss: 0.8413
Epoch: 005/010 | Batch 150/375 |Loss: 1.0527
Epoch: 005/010 | Batch 300/375 |Loss: 1.0130
Epoch: 00

In [9]:
print('Test accuracy: %.2f%%' % (metric_acc(model, test_loader)))

Test accuracy: 70.75%
